In [1]:
import sys
sys.path.append('..')

In [17]:
import pandas as pd
import numpy as np
import data
from lib.utils import iter_bucket

In [11]:
samples_age = pd.read_pickle('../data/geo-annotation/samples.age.cleaned.v1.pickle')
samples_tissue = (
    pd.read_pickle('../data/geo-annotation/samples.tissue.cleaned.v1.pickle')
    .reset_index()
    .rename(columns={'item': 'accession', 'collapsed': 'tissue_id'})
)
samples_norms = pd.read_pickle('../data/geo-annotation/samples.norms.07.res4.pickle').query('norm==1')

In [5]:
samples_age.head()

,accession,characteristics,series,platform,organism,age,isfloat
22869,E-MTAB-872_0,"{'Age': '0', 'CellType': 'embryonic stem', 'In...",[E-MTAB-872],A-GEOD-10558,Homo sapiens,0,True
22870,E-MTAB-872_1,"{'Age': '0', 'CellType': 'embryonic stem', 'In...",[E-MTAB-872],A-GEOD-10558,Homo sapiens,0,True
22871,E-MTAB-872_2,"{'Age': '0', 'CellType': 'embryonic stem', 'In...",[E-MTAB-872],A-GEOD-10558,Homo sapiens,0,True
22890,E-MTAB-487_0,"{'TimeUnit': 'years', 'Age': '0', 'CellType': ...",[E-MTAB-487],A-GEOD-8490,Homo sapiens,0,True
22891,E-MTAB-487_1,"{'TimeUnit': 'years', 'Age': '0', 'CellType': ...",[E-MTAB-487],A-GEOD-8490,Homo sapiens,0,True


In [9]:
samples_tissue.head()

,accession,tissue_id
0,GSM1006272,BTO:0000759
1,E-MTAB-3099_162,BTO:0000759
2,GSM645939,BTO:0000855
3,GSM429771,BTO:0000848
4,E-MTAB-3732_3998,BTO:0000876


In [12]:
samples_norms.head()

,accession,norm
0,GSM1422840,1
1,GSM1422847,1
2,GSM1422852,1
3,GSM1422846,1
4,GSM1422851,1


In [15]:
samples = (
    samples_tissue
    .merge(samples_norms, how='outer')
    .merge(samples_age, how='outer')
    [['accession', 'tissue_id', 'age', 'norm']]
)
samples.head()

,accession,tissue_id,age,norm
0,GSM1006272,BTO:0000759,NaN,NaN
1,E-MTAB-3099_162,BTO:0000759,NaN,NaN
2,GSM645939,BTO:0000855,52,NaN
3,GSM429771,BTO:0000848,NaN,NaN
4,E-MTAB-3732_3998,BTO:0000876,NaN,1


In [16]:
import pymongo as pm
db = pm.MongoClient().scraper_test_dev

In [28]:
from time import time

print('Annotating: #{}'.format(samples.shape[0]))
for bucket in iter_bucket(map(lambda r: r[1].to_dict(), samples.iterrows())):
    bulk = db.samples.initialize_unordered_bulk_op()
    print("#1" ,time())
    for item in bucket:
        accession = item['accession']
        annotation = dict((k, v) for k, v in item.items() if k != 'accession' and pd.notnull(v))
        
        bulk.find({'accession': accession}).update_one({'$set': {'annotation': annotation}})
    print("#2" ,time())
    res = bulk.execute()
    print("#3" ,time())
    print(res) 

Annotating: #1138556
#1 1445447089.0820115
#2 1445447089.1737633
#3 1445447089.6342466
{'writeErrors': [], 'nModified': 0, 'upserted': [], 'nUpserted': 0, 'nRemoved': 0, 'writeConcernErrors': [], 'nMatched': 3001, 'nInserted': 0}
#1 1445447089.9437294
#2 1445447090.0356326
#3 1445447092.514421
{'writeErrors': [], 'nModified': 0, 'upserted': [], 'nUpserted': 0, 'nRemoved': 0, 'writeConcernErrors': [], 'nMatched': 3001, 'nInserted': 0}
#1 1445447092.8215544
#2 1445447093.0359533
#3 1445447095.4774463
{'writeErrors': [], 'nModified': 0, 'upserted': [], 'nUpserted': 0, 'nRemoved': 0, 'writeConcernErrors': [], 'nMatched': 3001, 'nInserted': 0}
#1 1445447095.785655
#2 1445447095.877653
#3 1445447098.3440282
{'writeErrors': [], 'nModified': 0, 'upserted': [], 'nUpserted': 0, 'nRemoved': 0, 'writeConcernErrors': [], 'nMatched': 3001, 'nInserted': 0}
#1 1445447098.6568143
#2 1445447098.7504687
#3 1445447101.2213078
{'writeErrors': [], 'nModified': 0, 'upserted': [], 'nUpserted': 0, 'nRemoved': 